In [1]:
# Import necessary libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
from category_encoders import OneHotEncoder
from sklearn.pipeline import Pipeline
import joblib
from pathlib import Path

# Supress warnings
import warnings
warnings.simplefilter(action="ignore", category=Warning)

In [3]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define color scheme
colors = {
    "background": "#f9f9f9",  # Light gray background
    "text": "#333333",        # Dark gray text color
    "accent": "#4CAF50",      # Green accent color
    "button": "#FF5733",      # Orange button color
}

# html.Div(style={"marginBottom": "20px"}, children=[

app.layout = html.Div(style={"backgroundColor": colors["background"], "padding": "20px"}, children=[

    html.H1("Market Cap Prediction", style={"color": colors["text"], "textAlign": "center"}),

    # style={"color": "red", "marginTop": "10px"}

    html.Div(id="open-val-msg", style={"color": "red", "marginTop": "10px"}),
    html.Div(style={"marginBottom": "20px"}, children=[
        html.Label("Open: ", style={"color": colors["text"]}),
        dcc.Input(id="open-input", type="number", value=0),
    ]),

    html.Div(id="high-val-msg", style={"color": "red", "marginTop": "10px"}),
    html.Div(style={"marginBottom": "20px"}, children=[
        html.Label("High: ", style={"color": colors["text"]}),
        dcc.Input(id="high-input", type="number", value=0),
    ]),

    html.Div(id="low-val-msg", style={"color": "red", "marginTop": "10px"}),
    html.Div(style={"marginBottom": "20px"}, children=[
        html.Label("Low: ", style={"color": colors["text"]}),
        dcc.Input(id="low-input", type="number", value=0),
    ]),

    html.Div(id="close-val-msg", style={"color": "red", "marginTop": "10px"}),
    html.Div(style={"marginBottom": "20px"}, children=[
        html.Label("Close: ", style={"color": colors["text"]}),
        dcc.Input(id="close-input", type="number", value=0),
    ]),

    html.Div(id="volume-val-msg", style={"color": "red", "marginTop": "10px"}),
    html.Div([
        html.Label("Volume: ", style={"color": colors["text"]}),
        dcc.Input(id="volume-input", type="number", value=0),
    ]),
    
    html.Button("Predict", id="predict-button", n_clicks=0, style={"backgroundColor": colors["button"], "color": "#FFFFFF", "marginTop": "20px"}),
    html.Div(id="output-prediction", style={"marginTop": "20px", "fontWeight": "bold", "fontSize": "18px", "color": colors["accent"], "textAlign": "center"})
])

error_type = {
    "NoneType": "All inputs must have a value",
    "NegNum": "All numeric inputs must be non-negative.",
    "NAE": ""
}

@app.callback(
    Output("open-val-msg", "children"),
    Input("open-input", "value")
)
def validate_open(value):

    if value is None:
        return error_type["NoneType"]

    if value < 0:
        return error_type["NegNum"]

    return error_type["NAE"]


@app.callback(
    Output("high-val-msg", "children"),
    Input("high-input", "value")
)
def validate_high(value):

    if value is None:
        return error_type["NoneType"]

    if value < 0:
        return error_type["NegNum"]

    return error_type["NAE"]


@app.callback(
    Output("low-val-msg", "children"),
    Input("low-input", "value")
)
def validate_low(value):

    if value is None:
        return error_type["NoneType"]

    if value < 0:
        return error_type["NegNum"]

    return error_type["NAE"]

@app.callback(
    Output("close-val-msg", "children"),
    Input("close-input", "value")
)
def validate_close(value):

    if value is None:
        return error_type["NoneType"]

    if value < 0:
        return error_type["NegNum"]

    return error_type["NAE"]


@app.callback(
Output("volume-val-msg", "children"),
Input("volume-input", "value")
)
def validate_volume(value):

    if value is None:
        return error_type["NoneType"]

    if value < 0:
        return error_type["NegNum"]

    return error_type["NAE"]


# Load Model
model = joblib.load("../artifacts/model.sav")

# Define callback function to handle user input and generate prediction
@app.callback(
    Output("output-prediction", "children"),

    [Input("predict-button", "n_clicks")],
    [State("open-input", "value"),
    State("high-input", "value"),
    State("low-input", "value"),
    State("close-input", "value"),
    State("volume-input", "value"),]
)
def predict_market_cap(n_clicks, open, high, low, close, volume):
    
    input_data = pd.DataFrame({
        "Open": [open],
        "High":[high],
        "Low": [low],
        "Close": [close],
        "Volume": [volume]
    })
    
    # Make predictions
    prediction = model.predict(input_data)[0]*-1
    return f"Prediction: {prediction}"

In [4]:

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
